# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


In [ ]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
%matplotlib inline

In [ ]:
### Helper Functions

def plotImageFiles(filenames):
    for f in filenames:
        fig = plt.figure()
        fig.set_size_inches(3,6)
        img = mpimg.imread(f)
        plt.title(f)
        plt.axis('off')
        plt.imshow(img)
        
def plotImage(img, title=None, cmap=None):
    fig = plt.figure()
    fig.set_size_inches(4,8)
    if title is not None:
        plt.title(title)
    plt.axis('off')
    if cmap is None:
        plt.imshow(img)
    else:
        plt.imshow(img, cmap=cmap)
    
def plotMultipleImages(images, labels=None, ptitle=None, cmap=None):
    """ This function will plot the images specified in a
    single plot.
    """
    numImages = len(images)
    #fig = plt.figure(figsize=(3, 6*numImages))
    fig = plt.figure()
    if ptitle is not None:
        fig.suptitle(ptitle, fontsize="x-large")
    ii = 1
    for img in images:
        ax = fig.add_subplot(1, numImages, ii)
        if labels is not None:
            ax.set_title(labels[ii-1], fontsize="xx-small")
        ax.set_aspect(15)
        plt.axis('off')
        if cmap is not None:
            ax.imshow(img.squeeze(), cmap=cmap)
        else:
            ax.imshow(img.squeeze())
        ii += 1
        
def plotImageAndPolygon(img, polygonPts):
    fig, ax = plt.subplots(1)
    pgn = plt.Polygon(polygonPts, color='r', alpha=0.25, lw=2)
    ax.imshow(img)
    ax.add_patch(pgn)
    
def plotImageAndMultiplePolygons(img, multPolyPts1, multPolyPts2, cmap=None):
    fig, ax = plt.subplots(1)
    if cmap is None:
        ax.imshow(img)
    else:
        ax.imshow(img, cmap=cmap)
    for p in range(0, len(multPolyPts1)):
        pgn1 = plt.Polygon(multPolyPts1[p], color='r', alpha=0.25, lw=1)
        pgn2 = plt.Polygon(multPolyPts2[p], color='b', alpha=0.25, lw=1)
        ax.add_patch(pgn1)
        ax.add_patch(pgn2)
        
def plot3SideBySide(oimg, timg, hist):
    fig = plt.figure(figsize=(12,3))
    plt.subplot(131)
    plt.imshow(oimg)
    plt.subplot(132)
    plt.imshow(timg, cmap="gray")
    plt.subplot(133)
    plt.plot(hist)
    
def plotLaneFit(timg, llfit, rlfit):
    plt.figure()
    # Generate x and y values for plotting
    ploty = np.linspace(0, timg.shape[0]-1, timg.shape[0] )
    left_fitx = llfit[0]*ploty**2 + llfit[1]*ploty + llfit[2]
    right_fitx = rlfit[0]*ploty**2 + rlfit[1]*ploty + rlfit[2]
    plt.imshow(timg, cmap="gray")
    plt.plot(left_fitx, ploty, color='r', lw=3)
    plt.plot(right_fitx, ploty, color='b', lw=3)
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    
print("*** Helper Functions Defined ***")

### Camera Calibration

In [ ]:
### Camera Calibration

def calibrateCamera():
    imfiles = glob.glob('camera_cal/calibration*.jpg')
    nx = 9
    ny = 6
    #3d real world points
    objPoints = []
    #2d camera image points
    imgPoints = []
    #Corners returned by findChessboardCorners contains corners 
    #going left to right and then down. So we have to generate the
    #idealized objpoint to go in the same direction. We also assume
    #that the z-axis is 0 in the objpoint i.e., the image is placed
    #on a 2d plane
    objpoint = np.zeros((nx*ny, 3), np.float32)
    objpoint[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)
    for imfile in imfiles:
        img = mpimg.imread(imfile)
        #print(imfile)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
        if ret:
            imgPoints.append(corners)
            objPoints.append(objpoint)
    #ret, mtx, dist, rvecs, tvecs
    return cv2.calibrateCamera(objPoints, imgPoints, (gray.shape[1], gray.shape[0]), None, None)
 
def plotUndistortedImages():
    imfiles = glob.glob('camera_cal/calibration*.jpg')
    ret, mtx, dist, rvecs, tvecs = calibrateCamera()
    for imfile in imfiles:
        img = mpimg.imread(imfile)
        dst = cv2.undistort(img, mtx, dist, None, mtx)
        fig = plt.figure()
        fig.add_subplot(1, 2, 1)
        #plt.axis('off')
        plt.imshow(img)
        fig.add_subplot(1, 2, 2)
        plt.imshow(dst)

#plotUndistortedImages()
print("*** Camera Calibration Functions Defined ***")

### Gradient and Color Thresholding

In [ ]:
def getUndistortedImage(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)

def genericSobelThresholding(img, mfunc, ksize=3, thresh=(0,255)):
    # Grayscale transformation if the image has 3 channels
    # Otherwise assume it has already been transformed
    if img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply cv2.Sobel()
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=ksize)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=ksize)
    scaled_sobel = mfunc(sobelx, sobely)
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel > thresh[0]) & (scaled_sobel < thresh[1])] = 1
    return binary_output

def gradientAbsoluteThresholding(img, orient='x', ksize=3, thresh=(0,255)):
    def gradAbsSobel(sobelx, sobely):
        #Calculate scaled value of gradient
        if orient == 'x':
            sobel = sobelx
        else:
            sobel = sobely
        abs_sobel = np.absolute(sobel)
        scaled_sobel = np.uint8((abs_sobel * 255) / np.max(abs_sobel))
        return scaled_sobel
    return genericSobelThresholding(img, gradAbsSobel, ksize, thresh)

def gradientMagnitudeThresholding(img, ksize=3, thresh=(0,255)):
    def gradMagSobel(sobelx, sobely):
        #Calculate scaled magnitude of gradient
        gradmag = np.sqrt(sobelx**2 + sobely**2)
        scaled_gradmag = np.uint8((gradmag * 255) / np.max(gradmag))
        return scaled_gradmag
    return genericSobelThresholding(img, gradMagSobel, ksize, thresh)

def gradientDirectionThresholding(img, ksize=3, thresh=(0,np.pi/2)):
    def gradDirSobel(sobelx, sobely):
        #Calculate direction of gradient
        absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
        return absgraddir
    return genericSobelThresholding(img, gradDirSobel, ksize, thresh)

def colorThresholding(img, channel=0, colspace = cv2.COLOR_RGB2HLS, thresh=(0,255)):
    img = cv2.cvtColor(img, colspace)
    colchannel = img[:,:,channel]
    binary_output = np.zeros_like(colchannel)
    binary_output[(colchannel > thresh[0]) & (colchannel <= thresh[1])] = 1
    return binary_output

print("*** Gradient and Color Thresholding Functions Defined ***")

### Perspective Transformation

In [ ]:
# Compute the perspective transformation matrix using the straight line test images

def getPerspectiveTransformedImage(img, M):
    img_shape = (img.shape[1], img.shape[0])
    warped = cv2.warpPerspective(img, M, img_shape, flags=cv2.INTER_LINEAR)
    return warped

def getPerspectiveTransformationMatrix(inverse=False):
    #Manually identified source points for Image 2
    #bottom-left, bottom-right, top-right, top-left - (x, y)
    ## Other values that also work
    #srcpts2 = np.float32([(337, 633), (975, 633), (709, 463), (575, 463)])
    #dstpts2 = np.float32([(300, 720), (900, 720), (900, 0), (300, 0)])
    srcpts2 = np.float32([(337, 633), (975, 633), (709, 463), (575, 463)])
    dstpts2 = np.float32([(200, 720), (1000, 720), (1000, 0), (200, 0)])
    slfile2 = 'test_images/straight_lines2.jpg'
    if not inverse:
        M = cv2.getPerspectiveTransform(srcpts2, dstpts2)
    else:
        M = cv2.getPerspectiveTransform(dstpts2, srcpts2)
    return M

def testPerspectiveTransformation(M):
    #Manually identified source points for Image 1
    #bottom-left, bottom-right, top-right, top-left - (x, y)
    srcpts1 = np.float32([(331, 633), (973, 633), (702, 461), (581, 461)])
    dstpts1 = np.float32([(200, 720), (1000, 720), (1000, 0), (200, 0)])
    slfile1 = 'test_images/straight_lines1.jpg'
    origImg1 = mpimg.imread(slfile1)
    warpedImg1 = getPerspectiveTransformedImage(origImg1, M)
    plotImageAndPolygon(origImg1, srcpts1)
    plotImageAndPolygon(warpedImg1, dstpts1)
    
print("*** Perspective Transformation Functions Defined ***")


### Lane Detection

In [ ]:
def getFittedLanePixels(img_shape, llfit, rlfit):
    # Evenly spaced y-pixel values
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    #X values corresponding to the y-pixels
    leftfitx = llfit[0] * ploty ** 2 + llfit[1] * ploty + llfit[2]
    rightfitx = rlfit[0] * ploty ** 2 + rlfit[1] * ploty + rlfit[2]
    return ploty, leftfitx, rightfitx

def getRadiusOfCurvatureOfFitInPixels(img_shape, llfit, rlfit):
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0] )
    y_eval = np.max(ploty)
    llRadCurve = ((1 + (2 * llfit[0] * y_eval + llfit[1]) ** 2) ** 1.5) / np.absolute(2 * llfit[0])
    rlRadCurve = ((1 + (2 * rlfit[0] * y_eval + rlfit[1]) ** 2) ** 1.5) / np.absolute(2 * rlfit[0])
    return llRadCurve, rlRadCurve

def getRadiusOfCurvatureOfFitInMeters(img_shape, llfit, rlfit):
    # Assumed meters per pixel in our image each dimension
    ym_per_pix = 30/720
    xm_per_pix = 3.7/700
    ploty, leftfitx, rightfitx = getFittedLanePixels(img_shape, llfit, rlfit)
    y_eval = np.max(ploty)
    # Fit new polynomials to x,y in world space after using above conversion factors
    llFitMeters = np.polyfit(ploty * ym_per_pix, leftfitx * xm_per_pix, 2)
    rlFitMeters = np.polyfit(ploty * ym_per_pix, rightfitx * xm_per_pix, 2)
    # Calculate the new radii of curvature
    llRadCurve = ((1 + (2 * llFitMeters[0] * y_eval * ym_per_pix + llFitMeters[1]) ** 2) ** 1.5)
    llRadCurve /= np.absolute(2 * llFitMeters[0])
    rlRadCurve = ((1 + (2 * rlFitMeters[0] * y_eval * ym_per_pix + rlFitMeters[1]) ** 2) ** 1.5)
    rlRadCurve /= np.absolute(2 * rlFitMeters[0])
    # Now our radius of curvature is in meters
    return llRadCurve, rlRadCurve

def getVehicleDeviationFromLaneCenterInPixels(img_shape, llfit, rlfit):
    ploty, leftfitx, rightfitx = getFittedLanePixels(img_shape, llfit, rlfit)
    #Get a single pixel representing the left and right lanes
    #at the bottom of the image. This is where y = img.shape[0].
    maxy = np.argmax(ploty)
    llpix = leftfitx[maxy]
    rlpix = rightfitx[maxy]
    lanecenter = (rlpix - llpix) // 2
    camcenter = img_shape[1] // 2
    deviationFromCenterInPixels = camcenter - lanecenter
    return deviationFromCenterInPixels

def getVehicleDeviationFromLaneCenterInMeters(img_shape, llfit, rlfit):
    xm_per_pix = 3.7/700
    deviationFromCenterInPixels = getVehicleDeviationFromLaneCenterInPixels(img_shape, llfit, rlfit)
    return xm_per_pix * deviationFromCenterInPixels

def slidingWindowLaneDetect(timg, nwindows=9, winextent=100, winshiftthresh=300, enableplot = False):
    """ Detects lane lines using a sliding window that starts at the bottom of
    the given image and recenters itself based on lane curvature. It returns a
    tuple of polynomial fits for left lane and right lane respectively.
    """
    hist = np.sum(timg[timg.shape[0]/3:, :], axis = 0)
    midpt = hist.shape[0]//2
    llxbase = np.argmax(hist[0:midpt])
    rlxbase = np.argmax(hist[midpt:]) + midpt
    nz = timg.nonzero()
    nzx = np.array(nz[1])
    nzy = np.array(nz[0])    
    #Holds all the non-zero left lane pixels
    ll_pixels = []
    #Holds all the non-zero right lane pixels
    rl_pixels = []
    #Holds window corners for plotting
    llPolyPts = []
    rlPolyPts = []
    for i in range(0, nwindows):
        #Compute left lane sliding window corners' (x,y)
        ll_win_x_left = llxbase - winextent
        ll_win_x_right = llxbase + winextent
        ll_win_y_high = timg.shape[0] - i * (timg.shape[0]//nwindows)
        ll_win_y_low = ll_win_y_high - (timg.shape[0]//nwindows)       
        if enableplot:
            #bl, br, tr, tl
            ll_rect = [(ll_win_x_left, ll_win_y_low), (ll_win_x_right, ll_win_y_low),\
                       (ll_win_x_right, ll_win_y_high), (ll_win_x_left, ll_win_y_high)]
            llPolyPts.append(ll_rect)
        #Compute right lane sliding window corners' (x,y)
        rl_win_x_left = rlxbase - winextent
        rl_win_x_right = rlxbase + winextent
        rl_win_y_high = timg.shape[0] - i * (timg.shape[0]//nwindows)
        rl_win_y_low = rl_win_y_high - (timg.shape[0]//nwindows)
        if enableplot:
            #bl, br, tr, tl
            rl_rect = [(rl_win_x_left, rl_win_y_low), (rl_win_x_right, rl_win_y_low),\
                       (rl_win_x_right, rl_win_y_high), (rl_win_x_left, rl_win_y_high)]
            rlPolyPts.append(rl_rect)
        #Compute pixels that are non-zero within sliding window
        ll_win_pixels = ((nzx >= ll_win_x_left) & (nzx < ll_win_x_right)\
                                 & (nzy >= ll_win_y_low) & (nzy < ll_win_y_high)).nonzero()[0]
        ll_pixels.append(ll_win_pixels)
        
        rl_win_pixels = ((nzx >= rl_win_x_left) & (nzx < rl_win_x_right)\
                                 & (nzy >= rl_win_y_low) & (nzy < rl_win_y_high)).nonzero()[0]
        rl_pixels.append(rl_win_pixels)
        
        #Shift lane bases if we have a lot of non-zero pixels in this sliding window
        if len(ll_win_pixels) > winshiftthresh:
            llxbase = int(np.mean(nzx[ll_win_pixels]))
        if len(rl_win_pixels) > winshiftthresh:
            rlxbase = int(np.mean(nzx[rl_win_pixels]))
    
    if enableplot:
        plotImageAndMultiplePolygons(timg, llPolyPts, rlPolyPts, cmap="gray")
    
    ll_pixels = np.concatenate(ll_pixels)
    rl_pixels = np.concatenate(rl_pixels)
    llx = nzx[ll_pixels]
    lly = nzy[ll_pixels]
    rlx = nzx[rl_pixels]
    rly = nzy[rl_pixels]
    
    llfit = np.polyfit(lly, llx, 2)
    rlfit = np.polyfit(rly, rlx, 2)
    
    return llfit, rlfit


def targetedLaneDetect(warpedImage, prevllfit, prevrlfit):
    """ Detects lane lines on given image using pre-existing
    line fits from the previous video frame. It returns a
    tuple of polynomial fits for left lane and right lane respectively.
    """
    nz = warpedImage.nonzero()
    nzy = np.array(nz[0])
    nzx = np.array(nz[1])
    margin = 100
    ll_pixels = ((nzx > (prevllfit[0] * (nzy ** 2) + prevllfit[1] * nzy + prevllfit[2] - margin)) & \
                      (nzx < (prevllfit[0] * (nzy ** 2) + prevllfit[1] * nzy + prevllfit[2] + margin))) 
    rl_pixels = ((nzx > (prevrlfit[0] * (nzy ** 2) + prevrlfit[1] * nzy + prevrlfit[2] - margin)) & \
                       (nzx < (prevrlfit[0] * (nzy ** 2) + prevrlfit[1] * nzy + prevrlfit[2] + margin)))

    llx = nzx[ll_pixels]
    lly = nzy[ll_pixels] 
    rlx = nzx[rl_pixels]
    rly = nzy[rl_pixels]
    
    llfit = np.polyfit(lly, llx, 2)
    rlfit = np.polyfit(rly, rlx, 2)
    
    ll_c2_error = (llfit[0] - prevllfit[0]) / prevllfit[0]
    ll_c1_error = (llfit[1] - prevllfit[1]) / prevllfit[1]
    ll_c_error = (llfit[2] - prevllfit[2]) / prevllfit[2]
    
    rl_c2_error = (rlfit[0] - prevrlfit[0]) / prevrlfit[0]
    rl_c1_error = (rlfit[1] - prevrlfit[1]) / prevrlfit[1]
    rl_c_error = (rlfit[2] - prevrlfit[2]) / prevrlfit[2]
    
    CUTOFF = 20
    if abs(ll_c2_error) > CUTOFF or abs(ll_c1_error) > CUTOFF or abs(rl_c2_error) > CUTOFF or abs(rl_c1_error) > CUTOFF:
        #print("*** returning prev")
        return prevllfit, prevrlfit
    
    #ol = "{}, {}, {}, {}, {}, {}\n".format(ll_c2_error, ll_c1_error, ll_c_error, rl_c2_error, rl_c1_error, rl_c_error)
    #with open("errors.txt", "a") as of:
    #    of.write(ol)
    
    return llfit, rlfit


def getImageWithLane(warpedImg, undistImg, invPerspectiveTransMatrix, llfit, rlfit, llRocM, rlRocM, vPosM):
    ploty, leftfitx, rightfitx = getFittedLanePixels(warpedImg.shape, llfit, rlfit)
    # Create an image to draw the lines on
    zeroImg = np.zeros_like(warpedImg).astype(np.uint8)
    multiChanImg = np.dstack((zeroImg, zeroImg, zeroImg))
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([leftfitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([rightfitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(multiChanImg, np.int_([pts]), (0, 255, 0))
    # Warp the blank back to original image space using inverse perspective matrix
    newWarpedImg = cv2.warpPerspective(multiChanImg, invPerspectiveTransMatrix, (warpedImg.shape[1], warpedImg.shape[0])) 
    # Combine the result with the original undistorted image
    result = cv2.addWeighted(undistImg, 1, newWarpedImg, 0.3, 0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    llRocM = round(llRocM, 2)
    rlRocM = round(rlRocM, 2)
    vPosM = round(vPosM, 2)
    cv2.putText(result, "(LeftRoC, RightRoC, CarOffset) = ({}m, {}m, {}m)".format(llRocM, rlRocM, vPosM),\
                (int(result.shape[1] * 0.2), int(result.shape[0] * 0.05)),\
                font, 1, (255,255,255), 2, cv2.LINE_AA)
    return result

print("*** Lane Detection Functions Defined ***")


### Overall Pipeline

In [ ]:
def pipeline(img, distCoeffs, camMtx, perspTransMtx, invPerspTransMtx, prevllfit=None, prevrlfit=None):
    dimg = getUndistortedImage(img, camMtx, distCoeffs)
    colthreshimg_s = colorThresholding(dimg, channel=2, thresh=(180,255))
    absthreshimg = gradientAbsoluteThresholding(dimg, ksize=5, thresh=(20,80))
    #
    #Direction of gradient == +/- np.pi/2 is a vertical line.
    #We'll never have a fully vertical line at this point due to perspective.
    #So use something smaller than np.pi/2 as max
    #graddirthreshimg = gradientDirectionThresholding(dimg, ksize=13, thresh=(0.7,1.3))
    #gradmagthreshimg = gradientMagnitudeThresholding(dimg, ksize=5, thresh=(30,100))
    #
    combinedimg = np.zeros_like(colthreshimg_s)
    combinedimg[((colthreshimg_s == 1) | (absthreshimg == 1))] = 1
    #
    #images = [dimg, colthreshimg_s, absthreshimg, gradmagthreshimg, graddirthreshimg, combinedimg]
    #labels = ["Undistorted", "S-Channel Color", "Gradient AbsoluteX", "Gradient Magnitude", "Gradient Direction", "Combined"]
    #plotMultipleImages(images, labels, tfile, "gray")
    #
    warpedCombinedImage = getPerspectiveTransformedImage(combinedimg, perspTransMtx)
    if prevllfit is None or prevrlfit is None:
        llfit, rlfit = slidingWindowLaneDetect(warpedCombinedImage)
    else:
        llfit, rlfit = targetedLaneDetect(warpedCombinedImage, prevllfit, prevrlfit)
    #plotLaneFit(timg, llfit, rlfit)
    img_shape = warpedCombinedImage.shape
    #llRadCurvePixels, rlRadCurvePixels = getRadiusOfCurvatureOfFitInPixels(img_shape, llfit, rlfit)
    llRadCurveMeters, rlRadCurveMeters = getRadiusOfCurvatureOfFitInMeters(img_shape, llfit, rlfit)
    #
    vehicleDeviationFromLaneCenterInMeters = getVehicleDeviationFromLaneCenterInMeters(img_shape, llfit, rlfit)
    #
    imgWithLane = getImageWithLane(warpedCombinedImage, dimg, invPerspTransMtx,\
                                   llfit, rlfit, llRadCurveMeters, rlRadCurveMeters,\
                                   vehicleDeviationFromLaneCenterInMeters)
    #plotImage(imgWithLane)
    return imgWithLane, llfit, rlfit

In [ ]:
def testPipelineOnTestImages():
    ret, camMtx, distCoeffs, rvecs, tvecs = calibrateCamera()
    perspTransMtx = getPerspectiveTransformationMatrix()
    invPerspTransMtx = getPerspectiveTransformationMatrix(inverse=True)
    tfiles = glob.glob('test_images/test*.jpg')
    for tfile in tfiles:
        timg = mpimg.imread(tfile)
        imgWithLane, llfit, rlfit = pipeline(timg, distCoeffs, camMtx,\
                                             perspTransMtx, invPerspTransMtx)
        #plotImage(imgWithLane)
        plt.figure(figsize=(10, 12))
        plt.imshow(imgWithLane)

#testPipelineOnTestImages()


### Lane Detection on Video

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

ret, camMtx, distCoeffs, rvecs, tvecs = calibrateCamera()
perspTransMtx = getPerspectiveTransformationMatrix()
invPerspTransMtx = getPerspectiveTransformationMatrix(inverse=True)
prevllfit = None
prevrlfit = None
def process_image(image):
    global prevllfit
    global prevrlfit
    laneDetectionFunction = targetedLaneDetect
    imgWithLane, prevllfit, prevrlfit = pipeline(image, distCoeffs,\
                                                 camMtx, perspTransMtx,\
                                                 invPerspTransMtx, prevllfit, prevrlfit)
    return imgWithLane

outvideofile = 'project_video_ld.mp4'
pvideo = VideoFileClip("project_video.mp4")
ovideo = pvideo.fl_image(process_image)
%time ovideo.write_videofile(outvideofile, audio=False)

In [ ]:

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(outvideofile))